# Data Reading

In [208]:
import pandas as pd
import numpy as np

In [209]:
df_ohlc=pd.read_csv('data/BinanceTop10Raw.csv',header=[0, 1],index_col=0)
df_ohlc.index=pd.DatetimeIndex(df_ohlc.index)
df_Close=df_ohlc.swaplevel(axis=1)['Close']

# Predicting Market Movements with Hull Moving Averge

First we define an empty dataframe to contain the return of the strategy for each coin

In [210]:
HMA_Returns=pd.DataFrame()

we need to define some functions to use

In [211]:
def WMA(data,period):
    """Weighted Moving Average (WMA) a technical indicator

    Args:
        data (Series).
        period (int): length.

    Returns:
        Series: Seires with WMA values 
    """
    
    data=pd.Series(data)
    return data.rolling(period).apply(lambda x: ((np.arange(period)+1)*x).sum()/(np.arange(period)+1).sum(), raw=True)
    
def HMA(data,period):
    """Hull Moving Average (HMA) a technical indicator

    Args:
        data (Series).
        period (int): length.

    Returns:
        Series: Seires with HMA values 
    """
    return WMA((WMA(data, period//2)*2)-(WMA(data, period)), int(np.sqrt(period)))

## Backtesting.py


Next we will use `backtesting.py` library to backtest, optimise and exctract returns from

### Strategy definition

In [212]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest


class Hma_start(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 45
    n2 = 200
    def init(self):
        # Precompute the two moving averages
        self.ma1 = self.I(HMA, self.data.Close, self.n1)
        self.ma2 = self.I(HMA, self.data.Close, self.n2)
    def next(self):

        if crossover(self.ma1, self.ma2) :
            if not self.position:
                self.buy()
            else:
                self.position.close()
                self.buy()
                
        elif False:
            pass
        elif crossover(self.ma2, self.ma1):
            if self.position:
                self.position.close()
                self.sell()


### Metric definition

In [213]:
def Metric_to_optimse(series):
    return series['Sharpe Ratio']

### Spliting the data to test, train

In [224]:
df_train,df_test=df_ohlc.loc[:'2022-09-01'],df_ohlc.loc['2022-09-01':]


In [215]:
instruments=df_ohlc.columns.get_level_values(0).unique()
parameters={}

for instrument in instruments:
    data = df_train[instrument]
    bt = Backtest(data, Hma_start , cash=1000000)
    stats = bt.optimize(n1=range(30, 250, 15),
                        n2=range(30, 250, 15),
                        maximize=Metric_to_optimse,
                        method='skopt',
                        constraint=lambda param: param.n1 < param.n2,
                        max_tries=100,
                        )._strategy
    params={'n1':stats.n1,'n2':stats.n2}
    parameters[instrument]=params

    

In [216]:
for instrument in instruments:
    data = df_test[instrument]
    HMA_Returns[instrument]=Backtest(data, Hma_start , cash=1000000).run(**parameters[instrument])._equity_curve.Equity/1000000

In [232]:
HMA_Returns.to_csv('ModelsResult/OptimizedHMA.csv')
parameters

{'BTCUSDT': {'n1': 47, 'n2': 71},
 'ETHUSDT': {'n1': 42, 'n2': 64},
 'BNBUSDT': {'n1': 76, 'n2': 149},
 'BUSDUSDT': {'n1': 180, 'n2': 227},
 'XRPUSDT': {'n1': 53, 'n2': 222},
 'DOGEUSDT': {'n1': 37, 'n2': 107},
 'ADAUSDT': {'n1': 62, 'n2': 218},
 'MATICUSDT': {'n1': 31, 'n2': 64},
 'DOTUSDT': {'n1': 30, 'n2': 45},
 'LTCUSDT': {'n1': 32, 'n2': 65}}

# Predicting Market Movements with Machine Learning